In [35]:

import sys
print(sys.executable)

checkpoint_dir = '../../.checkpoints/'

import torch
print(torch.__version__)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)


/home/thomas/crosscoders-feature-interactions/.venv/bin/python
2.5.1+cu124
cuda


In [36]:
from model_diffing.analysis.ft_analysis_util import load_crosscoder
from model_diffing.analysis.feature_graphing import plot_ft_rotations

In [37]:
# checkpoints
checkpoint_S = 'crosscoder_S/model_epoch_20.pt'
checkpoint_D = 'crosscoder_D/model_epoch_1.pt'
checkpoint_M = 'crosscoder_M/model_epoch_3.pt'
checkpoint_MF = 'crosscoder_MF/model_epoch_1.pt'
checkpoint_DF = 'crosscoder_DF/model_epoch_1.pt'

In [38]:
from model_diffing.scripts.train_topk_sleeper.run import TopkActivation

# load crosscoder decoder features
crosscoder_S, crosscoder_D, crosscoder_M, crosscoder_MF, crosscoder_DF = [
    load_crosscoder(checkpoint_dir + checkpoint, TopkActivation(k=20), DEVICE)
    for checkpoint in [checkpoint_S, checkpoint_D, checkpoint_M, checkpoint_MF, checkpoint_DF]
]

features_S, features_D, features_M, features_MF, features_DF = [
    crosscoder.W_dec_HMLD.reshape(crosscoder.W_dec_HMLD.shape[0], -1)
    for crosscoder in [crosscoder_S, crosscoder_D, crosscoder_M, crosscoder_MF, crosscoder_DF]
]

/home/thomas/crosscoders-feature-interactions/model_diffing/analysis/ft_analysis_util.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_

In [39]:
S_D_cosine_sims = compute_cosine_similarities(features_S, features_D)
S_M_cosine_sims = compute_cosine_similarities(features_S, features_M)
S_MF_cosine_sims = compute_cosine_similarities(features_S, features_MF)
S_DF_cosine_sims = compute_cosine_similarities(features_S, features_DF)
D_DF_cosine_sims = compute_cosine_similarities(features_D, features_DF)
M_MF_cosine_sims = compute_cosine_similarities(features_M, features_MF)

In [40]:
# print shapes
print(D_DF_cosine_sims.shape)

from bokeh.io import output_notebook
output_notebook()
# plot feature cosine similarities
plot_ft_rotations(D_DF_cosine_sims, M_MF_cosine_sims, 'D -> DF cosines similarities', 'M -> MF cosines similarities')

(1536,)


Loading BokehJS ...